# **Lithology Boundary Definition using CWT**

Github : https://github.com/galena100/Transform2020

<img src="https://raw.githubusercontent.com/galena100/Transform2020/master/t20-litho_boundary_from_gamma/test_on_synthetic_data.JPG" alt="Your image title" width="500"/>

We created this image from a synthetic GR log as part of this project.  It show the potential for finding lithology boundaries from geophysical logs.

 

Based on "***Improving Automated Geological Logging of Drill Holes by Incorporating Multiscale Spatial Methods***" works by [Evelyn Jun Hill](https://www.researchgate.net/publication/339871641_Improving_Automated_Geological_Logging_of_Drill_Holes_by_Incorporating_Multiscale_Spatial_Methods). 


**1. What did you guys do?**
---
Estimate lithology boundaries from a downhole geophysical log, using a continuous wavelet transform (CWT).



## **2. Who are you guys?**

- **Jared Armstrong** :
 
  "*I love working at the intersection of mining coal geology and Database Technology.  Free your data to dissolve your difficulties*"

  E-mail : jared@wholesolutions.com.au

  Slack : @Jared 

- **Leo C. Dinendra** : 

  "*A geophysicist DM worker in an oil company. I love for-loops and will fight for it.*"

  E-mail : leocd91@gmail.com , [LinkedIn](https://www.linkedin.com/in/leo-c-0988727b/)

  Slack : @leo 

- **Martin Bentley** :

  "*Messing about with computers and rocks. Accidental geophysicist.*"

  E-mail : astonmartin.bentley@gmail.com

  Slack : @mtb-za


**3. How you guys do it?**
---

Run all of the cells until the **Making Things Happen** heading.

### Standard Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import pywt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

## Defining our functions

### Importing data and getting our lithology set up.

In [ ]:
def extract_data(fname, borehole_id='test001', data_type='DENB_g_cc'):
    '''
    Read a CSV file. Extract only data for `borehole` in dataset, then use only `data_type` for further plots.
    returns:
        data: series of interest
        curve: Sav-Gol filtered dataset
    '''
    raw_data = pd.read_csv(fname)
    bore = raw_data.loc[raw_data['HOLEID'] == borehole_id].copy() # get data for the requested hole.
    bore.dropna(subset=[data_type], inplace=True)
    bore['DEPTH'] = bore['DEPTH'] * -1 #  Make depths positive down.
    bore['savgol'] = savgol_filter(bore[data_type], 21, 2)                                      #getting the GR data and filter using savgol
    
    return bore, borehole_id, data_type

In [ ]:
# We need a dict to convert lithologies to colours.
lith_colours = {
    'Basalt': 'purple',
    'Calcite': 'lightgrey',
    'Carbonaceous Mudstone': 'lightgreen',
    'Clay': 'green',
    'Claystone': 'darkgreen',
    'Coal': 'black',
    'Core Loss': 'white',
    'Mudstone': 'darkgrey',
    'Sand': 'gold',
    'Sandstone': 'orange',
    'Siderite': 'lightblue',
    'Siltstone': 'brown',
    'Soil': 'chocolate',
    'Tuff': 'yellow',
    'Intermediate Intrusive': 'rebeccapurple',
    'Pyrite': 'darkgoldenrod',
}

In [ ]:
def make_lithology(borehole='test001', lithcolours=lith_colours):
    '''
    Using the provided lithology and lookup table, will give back a lithology table
    and list of colours.
    
    Needs a dictionary of lithologies with associated colours.
    '''
    lithology = pd.read_csv('t20-litho_boundary_from_gamma/2_litho.csv')
    lithology_lookup = pd.read_csv('t20-litho_boundary_from_gamma/2_Lookup_Table.csv')
    descriptions = lithology_lookup.to_dict()['DESCRIPTION'].values()
    lookups = lithology_lookup.to_dict()['LOOKUP'].values()

    lithology_lookup = {}
    for lookup, description in zip(lookups, descriptions):
        lithology_lookup[lookup] = description
    lithology.replace(to_replace=lithology_lookup, value=None, inplace=True)
    litho_first_hole = lithology[lithology['HOLEID'] == borehole].copy()
    litho_first_hole['spans'] = litho_first_hole['GEOLTO'] - litho_first_hole['GEOLFROM']
    
    litho_first_hole['colour'] = litho_first_hole['VALUE']
    litho_first_hole['colour'].replace(lith_colours, value=None, inplace=True)
    
    colours = litho_first_hole['colour'].to_list()
    
    return litho_first_hole, colours

### CWT, Boundary Definition, and Boundary Strength Computation

#### These are support functions, we need them for the data we plot.

In [ ]:
def resample_curve(curve, step):
    '''Returns a pandas series, keeping every `step`th item.'''
    return curve.iloc[list(range(0, len(curve), step))]

In [ ]:
def make_wavelet(curve, scale, wavelet_type, dt):
    '''
    Calculate the Continuous Wavelet Transform for a given CWT
    Returns:
        Transposed coefficients of the CWT
    '''
    [cfs, frequencies] = pywt.cwt(curve, scale, wavelet_type, dt)
    return cfs.T, frequencies

In [ ]:
def calculate_cwt(curve, dt, fscales=np.arange(1, 100,1)):
    '''Return coefficients of first and second order Gaussian CWT of given curve'''
    zcwt2, g2_freq = make_wavelet(curve, fscales, 'gaus2', dt)
    zcwt1, g1_freq = make_wavelet(curve, fscales, 'gaus1', dt)
    
    return zcwt2, zcwt1, g2_freq, g1_freq

In [ ]:
def make_boundary_strength(cs, boundary_threshold=30):
    '''
    Generates a horizontal histogram of depths where the CWT is equal to zero. 
    '''
    cs_allsegs = np.array(cs.allsegs[0])

    #print(cs_allsegs[0][:])

    #'''
    y_counts = []
    for cs_segs in cs_allsegs:
        if cs_segs.shape[0] > boundary_threshold: #  We can use this for additional filtering to find boundaries at different wavelengths.
            for seg in cs_segs:
                y_counts.append(seg[1])
    return y_counts

In [ ]:
def plotting_function(borehole, plot_type, lithology,
                      boundary_threshold=30,
                      fscales=np.arange(1, 100,1),
                      resample_curves=True,
                      y_top=0, y_bot=100,
                      borehole_id='A Borehole'):
    # Useful variables:
    dep = borehole[borehole['DEPTH'] > y_top]
    dep = dep[borehole['DEPTH'] < y_bot]['DEPTH']
    curve = borehole[borehole['DEPTH'] > y_top]
    curve = curve[borehole['DEPTH'] < y_bot]['savgol']
    plot_range = [y_top, y_bot]
    
    if resample_curves:
        dep_resampled = resample_curve(dep, 10)
        curve_resampled = resample_curve(curve, 10)
    else:
        dep_resampled = dep
        curve_resampled = curve

    dt = dep_resampled.iloc[1] - dep_resampled.iloc[0]

    zcwt, zcwt2, freq1, freq2 = calculate_cwt(curve_resampled, dt)
    
    print('Making plots.')
    # Begin plot
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 10))

    # Data for each plot
    # Base curve
    ax1.plot(curve_resampled, dep_resampled)
    # CWT
    cs=ax2.contour(fscales, dep_resampled, zcwt, levels=[0.0], colors='k', extend='both')
    print('Calculating boundary strengths.')
    
    ## We need to calculate our boundary strengths
    y_counts = make_boundary_strength(cs, boundary_threshold)
    
    ## Now carry on plotting.
    ax2.contourf(fscales, dep_resampled, zcwt,
                 levels=np.arange(np.min(zcwt), np.max(zcwt), 0.5), 
                 cmap="RdBu_r", extend='both')
    # Boundary strength
    ax3.hist(np.array(y_counts), bins=int(np.floor(dep.max())), orientation='horizontal', color='k')
    # Lithology
    ax4.barh(lithology['GEOLFROM'], align='edge',
             height=list(lithology['spans']),
             width=10, color=lithology['colour'],)

    # Set plot limits
    ax1.set_ylim(plot_range)
    ax2.set_ylim(plot_range)
    ax3.set_ylim(plot_range)
    ax4.set_ylim(plot_range)
    ax4.set_xlim([0.0,1.0])

    # Invert axes
    ax1.invert_yaxis()
    ax2.invert_yaxis()
    ax3.invert_yaxis()
    ax4.invert_yaxis()

    # Title
    ax1.set_xlabel(f'Smoothed {plot_type}')
    ax2.set_title(f'CWT from {np.round(np.min(freq1),2)} Hz to {np.round(np.max(freq2),2)} Hz')
    ax3.set_title('Boundary Strength')
    ax4.set_title('Lithology')

    # Axis Labels
    ax1.set_ylabel('Depth (m)')
    ax2.xaxis.set_visible(False)
    ax2.yaxis.set_visible(False)
    ax3.xaxis.set_visible(False)
    ax3.yaxis.set_visible(False)
    ax3.set_xlabel('Boundary Strength')
    ax4.xaxis.set_visible(False)
    #ax4.yaxis.set_visible(False) #  Keeping this visible, because the spine length goes weird sometimes.

    ax4.set_aspect(0.05)
    
    plt.title(f"CWT of {plot_type} in {borehole_id}")
    
    # Adding gridlines
    ax1.grid(zorder=1)
    ax2.grid(zorder=4);ax2.set_axisbelow(True)
    ax3.grid(zorder=4);ax3.set_axisbelow(True)
    ax4.grid(zorder=4)

## Making things happen:

Run the next cell, and choose your data. There are three boreholes, and four well curves to choose from.

In [ ]:
fname = 't20-litho_boundary_from_gamma/2_geophys.csv'
method = 'MC2F_us_f'
borehole_id = 'test001'
select_data = interactive(extract_data,
                          borehole_id=['test001', 'test002', 'test003'],
                          data_type=['GRDE_gapi', 'DENB_g_cc', 'MC2F_us_f', 'CADE_mm'],
                          fname=fixed(fname))
display(select_data)

Now that you have chosen your data, set up some of the plot's parameters.

In [ ]:
borehole = select_data.result[0]

y_top_w = widgets.IntSlider(value=int(borehole['DEPTH'].min()),
                            min=int(borehole['DEPTH'].min()),
                            max=int(borehole['DEPTH'].max()-5),
                            step=5, description='Window Top')
y_bot_w = widgets.IntSlider(value=int(borehole['DEPTH'].max()-5),
                            min=5, max=int(borehole['DEPTH'].max()),
                            step=5, description='Window Bottom')
threshold_slider = widgets.IntSlider(value=30, min=0, max=100, step=5, description='Boundary Threshold')

y_top_w_link = widgets.dlink((y_top_w, 'value'), (y_bot_w, 'min'))
y_bot_w_link = widgets.dlink((y_bot_w, 'value'), (y_top_w, 'max'))

You can tweak the sensitivity of the edge detection here, by changing the 'Boundary Threshold'. The 'Window Top' and 'Window Bottom' will let you zoom in on sections of the borehole. Sometimes the lithology plot goes a little screwy when doing this. No idea why.

If you want a different curve, or a different borehole, go back to the selection cell, then run the cell above and below this text again. Have fun!

In [ ]:
lithology, colours = make_lithology(borehole=borehole_id)
plot_data = interact_manual(plotting_function,
                            borehole=fixed(select_data.result[0]), 
                            plot_type=fixed(select_data.result[2]), 
                            lithology=fixed(lithology),
                            boundary_threshold=threshold_slider,
                            fscales=fixed(np.arange(1, 100,1)),
                            resample_curves=True,
                            y_top=y_top_w,
                            y_bot=y_bot_w,
                            borehole_id=fixed(borehole['HOLEID'].iloc[0]),
                            )